### Get Coins with < $100 Million Markt Cap  from Binance API and save to a csv file

In [4]:
import os 
from binance.client import Client 
import pandas as pd 
from dotenv  import  load_dotenv 

load_dotenv()

API_KEY = os.getenv('BINANCE_API_KEY')
API_SECRET = os.getenv('BINANCE_API_SECRET')

client  = Client(API_KEY,API_SECRET)

def get_small_cap_coins(market_cap_threshold=100_000_000):
    """
    Fetch recently listed coins with market cap under the specified threshold.
    """
    #  all tickers
    tickers = client.get_all_tickers()
    
    # (market cap under $100M)
    small_cap_coins = []
    for ticker in tickers:
        symbol = ticker['symbol']
        if symbol.endswith('USDT'):  
            try:
                # 24hr ticker data for market cap calculation
                ticker_24hr = client.get_ticker(symbol=symbol)
                volume = float(ticker_24hr['quoteVolume'])
                price = float(ticker_24hr['lastPrice'])
                market_cap = volume * price
                
                if market_cap < market_cap_threshold:
                    small_cap_coins.append(symbol)
            except Exception as e:
                print(f"Error processing {symbol}: {e}")
    
    return small_cap_coins

def fetch_price_volume_data(symbol, interval='5m', limit=60):
    """
    Fetch OHLC and volume data for a given symbol.
    """
    try:
        klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
        return [{
            'timestamp': kline[0],
            'open': float(kline[1]),
            'high': float(kline[2]),
            'low': float(kline[3]),
            'close': float(kline[4]),
            'volume': float(kline[5])
        } for kline in klines]
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None


if __name__ == "__main__":
  
    small_cap_coins = get_small_cap_coins()
    print(f"Small-cap coins: {small_cap_coins}")

    small_cap_df = pd.DataFrame(small_cap_coins , columns=['Symbol'])
    small_cap_df.to_csv('small_cap_coins.csv', index = False)
    
    if small_cap_coins:
        coin_data = fetch_price_volume_data(small_cap_coins[0])
        if coin_data:
            coin_df = pd.DataFrame(coin_data)
            coin_df.to_csv(f'{small_cap_coins[0]}_price_volume_data.csv', index=False)
        print(f"Sample data for {small_cap_coins[0]}: {coin_data[:1]}")



Small-cap coins: ['BCCUSDT', 'QTUMUSDT', 'EOSUSDT', 'TUSDUSDT', 'IOTAUSDT', 'ONTUSDT', 'TRXUSDT', 'ICXUSDT', 'VENUSDT', 'NULSUSDT', 'VETUSDT', 'PAXUSDT', 'BCHABCUSDT', 'BCHSVUSDT', 'WAVESUSDT', 'BTTUSDT', 'USDSUSDT', 'ONGUSDT', 'HOTUSDT', 'ZILUSDT', 'ZRXUSDT', 'FETUSDT', 'BATUSDT', 'XMRUSDT', 'IOSTUSDT', 'CELRUSDT', 'NANOUSDT', 'OMGUSDT', 'THETAUSDT', 'ENJUSDT', 'MITHUSDT', 'MATICUSDT', 'TFUELUSDT', 'ONEUSDT', 'FTMUSDT', 'ALGOUSDT', 'USDSBUSDT', 'GTOUSDT', 'ERDUSDT', 'DUSKUSDT', 'ANKRUSDT', 'WINUSDT', 'COSUSDT', 'NPXSUSDT', 'COCOSUSDT', 'MTLUSDT', 'TOMOUSDT', 'PERLUSDT', 'DENTUSDT', 'MFTUSDT', 'KEYUSDT', 'STORMUSDT', 'DOCKUSDT', 'WANUSDT', 'FUNUSDT', 'CVCUSDT', 'CHZUSDT', 'BANDUSDT', 'BUSDUSDT', 'BEAMUSDT', 'XTZUSDT', 'RENUSDT', 'RVNUSDT', 'HCUSDT', 'HBARUSDT', 'NKNUSDT', 'STXUSDT', 'KAVAUSDT', 'ARPAUSDT', 'IOTXUSDT', 'RLCUSDT', 'MCOUSDT', 'CTXCUSDT', 'TROYUSDT', 'VITEUSDT', 'FTTUSDT', 'EURUSDT', 'OGNUSDT', 'DREPUSDT', 'BULLUSDT', 'BEARUSDT', 'ETHBULLUSDT', 'ETHBEARUSDT', 'TCTUSDT', 'W

### Improved Code and added progression bar   for script

In [ ]:
import os
from binance.client import Client
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

# Load environment variables
load_dotenv()

API_KEY = os.getenv('BINANCE_API_KEY')
API_SECRET = os.getenv('BINANCE_API_SECRET')

client = Client(API_KEY, API_SECRET)

def get_filtered_symbols(market_cap_threshold=100_000_000, min_volume=1_000_000, min_days_listed=90):
    """
    Fetch USDT trading pairs with specific conditions:
      - Market cap below the threshold.
      - Minimum 24-hour trading volume.
      - Listed for at least the specified number of days.

    Args:
        market_cap_threshold (float): Maximum market cap.
        min_volume (float): Minimum 24-hour trading volume.
        min_days_listed (int): Minimum number of days listed.

    Returns:
        list: Filtered symbols meeting the criteria.
    """
    print("Fetching exchange info...")
    exchange_info = client.get_exchange_info()
    symbols = [
        symbol_info['symbol'] for symbol_info in exchange_info['symbols']
        if symbol_info['symbol'].endswith('USDT') and symbol_info['status'] == 'TRADING'
    ]

    filtered_symbols = []
    print("Processing symbols...")
    for symbol in tqdm(symbols):
        try:
            ticker_24hr = client.get_ticker(symbol=symbol)
            volume = float(ticker_24hr['quoteVolume'])
            price = float(ticker_24hr['lastPrice'])
            market_cap = volume * price

            # Fetch listing age (minimum 90 daily klines required)
            klines = client.get_klines(symbol=symbol, interval='1d', limit=min_days_listed)

            if (
                market_cap < market_cap_threshold
                and volume >= min_volume
                and len(klines) >= min_days_listed
            ):
                filtered_symbols.append({
                    'symbol': symbol,
                    'market_cap': market_cap,
                    'volume': volume,
                    'days_listed': len(klines)
                })
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    return sorted(filtered_symbols, key=lambda x: x['market_cap'])

def fetch_historical_data(symbol, interval='15m', days=14):
    """
    Fetch historical OHLC and volume data for a given symbol.

    Args:
        symbol (str): The cryptocurrency trading pair symbol (e.g., BTCUSDT).
        interval (str): The time interval for the kline data (e.g., '15m').
        days (int): Number of days to fetch data for.

    Returns:
        pd.DataFrame: DataFrame containing OHLC and volume data.
    """
    try:
        total_bars = days * 96  # 96 bars per day for 15m intervals
        print(f"Fetching historical data for {symbol} ({interval}, {days} days)...")
        klines = client.get_klines(symbol=symbol, interval=interval, limit=total_bars)

        data = [{
            'timestamp': pd.to_datetime(kline[0], unit='ms'),
            'open': float(kline[1]),
            'high': float(kline[2]),
            'low': float(kline[3]),
            'close': float(kline[4]),
            'volume': float(kline[5])
        } for kline in klines]

        return pd.DataFrame(data)

    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return pd.DataFrame()

def save_to_csv(data, filename):
    """
    Save a list of dictionaries or DataFrame to a CSV file.

    Args:
        data (list or pd.DataFrame): Data to save.
        filename (str): File name for the CSV.
    """
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data

    df.to_csv(filename, index=False)
    print(f"Saved data to {filename}.")

if __name__ == "__main__":
    # Step 1: Filter symbols
    filtered_symbols = get_filtered_symbols()

    if not filtered_symbols:
        print("No symbols met the criteria.")
    else:
        print(f"Filtered symbols found: {len(filtered_symbols)}")
        save_to_csv(filtered_symbols, "filtered_symbols.csv")

        # Step 2: Fetch historical data for each filtered symbol
        for symbol_info in filtered_symbols:
            symbol_name = symbol_info['symbol']
            historical_data = fetch_historical_data(symbol_name, interval='15m', days=14)

            if not historical_data.empty:
                filename = f"{symbol_name}_15m_data.csv"
                save_to_csv(historical_data, filename)
            else:
                print(f"No historical data found for {symbol_name}.")


### get the circulating supply for every symbol you fetch from the Binance API and then use the CoinGecko API  total ciruclating supply to get market cap 

In [ ]:


import requests

import concurrent.futures




def get_binance_symbols():
    """Fetch all trading pairs from Binance."""
    exchange_info = binance_client.get_exchange_info()
    symbols = [symbol_info['symbol'] for symbol_info in exchange_info['symbols'] if symbol_info['status'] == 'TRADING']
    return symbols

def get_price(symbol):
    """Get current price for a given symbol from Binance."""
    try:
        ticker = binance_client.get_ticker(symbol=symbol)
        return float(ticker['lastPrice'])
    except Exception as e:
        print(f"Error fetching price for {symbol}: {e}")
        return None

def get_circulating_supply(symbol):
    """Fetch circulating supply from CoinGecko."""
    coingecko_url = f"https://api.coingecko.com/api/v3/simple/price?ids={symbol}&vs_currencies=usd&include_market_cap=true"
    try:
        response = requests.get(coingecko_url)
        data = response.json()
        return data[symbol]['market_cap']
    except Exception as e:
        print(f"Error fetching circulating supply for {symbol}: {e}")
        return None

def calculate_market_cap(symbol):
    """Calculate market cap using price and circulating supply."""
    # Remove USDT from the symbol
    base_symbol = symbol.replace('USDT', '')
    
    price = get_price(symbol)
    circulating_supply = get_circulating_supply(base_symbol)
    
    if price is not None and circulating_supply is not None:
        market_cap = circulating_supply * price
        return symbol, market_cap
    return symbol, None

def main():
    symbols = get_binance_symbols()
    
    # Filter USDT pairs
    usdt_symbols = [symbol for symbol in symbols if symbol.endswith('USDT')]
    
    # Use ThreadPoolExecutor to fetch data in parallel
    with concurrent.futures.ThreadPoolExecutor() as executor:
        market_caps = list(executor.map(calculate_market_cap, usdt_symbols))
    
    # Filter out None values and create DataFrame
    valid_market_caps = [(symbol, cap) for symbol, cap in market_caps if cap is not None]
    
    # Create a DataFrame and save to CSV
    # df = pd.DataFrame(valid_market_caps, columns=['Symbol', 'Market Cap'])
    # df.to_csv('market_caps.csv', index=False)
    
    # print("Market caps calculated and saved to market_caps.csv")

if __name__ == "__main__":
    main()



### Binance API doesnt support giving market Cap info directly so i am using  2 weeks trading volume to approximate market cap  and it seems binance does not provide the date a coin was  listed so i would be fetching  historical 90 days of trading .

In [21]:
from datetime import datetime, timedelta
import concurrent.futures
import pandas as pd
from binance.client import Client
from dotenv import load_dotenv
import os 
from tqdm import tqdm

load_dotenv()

API_KEY = os.getenv('BINANCE_API_KEY')
API_SECRET = os.getenv('BINANCE_API_SECRET')

client = Client(API_KEY, API_SECRET)



def get_filtered_symbols_parallel(market_cap_threshold=100_000_000, min_volume=1_000_000, min_days_listed=90):
    """
    Fetch USDT trading pairs with specific conditions using parallel processing:
      - Market cap below the threshold.
      - Minimum 24-hour trading volume.
      - Listed for at least the specified number of days.

    Args:
        market_cap_threshold (float): Maximum market cap.
        min_volume (float): Minimum 24-hour trading volume.
        min_days_listed (int): Minimum number of days listed.

    Returns:
        list: Filtered symbols meeting the criteria.
    """
    print("Fetching exchange info...")
    exchange_info = client.get_exchange_info()
    symbols = [
        symbol_info['symbol'] for symbol_info in exchange_info['symbols']
        if symbol_info['symbol'].endswith('USDT') and symbol_info['status'] == 'TRADING'
    ]

    def process_symbol(symbol):
        try:
            ticker_24hr = client.get_ticker(symbol=symbol)
            volume = float(ticker_24hr['quoteVolume'])
            price = float(ticker_24hr['lastPrice'])
            market_cap = volume * price

            # Fetch listing age (minimum 90 daily klines required)
            klines = client.get_klines(symbol=symbol, interval='1d', limit=min_days_listed)

            if (
                market_cap < market_cap_threshold
                and volume >= min_volume
                and len(klines) >= min_days_listed
            ):
                return {
                    'symbol': symbol,
                    'market_cap': market_cap,
                    'volume': volume,
                    'days_listed': len(klines)
                }
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
        return None

    print("Processing symbols in parallel...")
    with ThreadPoolExecutor() as executor:
        results = list(tqdm(executor.map(process_symbol, symbols), total=len(symbols), desc="Processing symbols"))

    filtered_symbols = [result for result in results if result is not None]
    return sorted(filtered_symbols, key=lambda x: x['market_cap'])

# def get_filtered_symbols(market_cap_threshold=100_000_000, min_volume=1_000_000, min_days_listed=90 ,  batch_size=1500):
#     print("Fetching exchange info...")
#     exchange_info = client.get_exchange_info()
#     symbols = [
#         symbol_info['symbol'] for symbol_info in exchange_info['symbols']
#         if symbol_info['symbol'].endswith('USDT') and symbol_info['status'] == 'TRADING'
#     ]

#     filtered_symbols = []
#     print("Processing symbols in batches...")
#     for i in tqdm(range(0, len(symbols), batch_size), desc="Batch processing"):
#         batch = symbols[i:i + batch_size]
#         for symbol in batch:
#             try:
#                 ticker_24hr = client.get_ticker(symbol=symbol)
#                 volume = float(ticker_24hr['quoteVolume'])
#                 price = float(ticker_24hr['lastPrice'])
#                 market_cap = volume * price

#                 # Fetch listing age (minimum 90 daily klines required)
#                 klines = client.get_klines(symbol=symbol, interval='1d', limit=min_days_listed)

#                 if (
#                     market_cap < market_cap_threshold
#                     and volume >= min_volume
#                     and len(klines) >= min_days_listed
#                 ):
#                     filtered_symbols.append({
#                         'symbol': symbol,
#                         'market_cap': market_cap,
#                         'volume': volume,
#                         'days_listed': len(klines)
#                     })
#             except Exception as e:
#                 print(f"Error processing {symbol}: {e}")

#     return sorted(filtered_symbols, key=lambda x: x['market_cap'])

# def calculate_market_cap(symbol, price, volume):
#     """Calculate market cap using price and 14-day volume."""
#     if price is not None and volume is not None:
#         market_cap = price * volume
#         return symbol, market_cap
#     return symbol, None

def main():
    
    filtered_symbols = get_filtered_symbols_parallel()

    if not filtered_symbols:
        print("No symbols met the criteria.")
    else:
        print(f"Filtered symbols found: {len(filtered_symbols)}")
        
        # Print out the processed symbols
        for symbol_data in filtered_symbols:
            print(symbol_data)
    
    
    # Filter USDT pairs
    # usdt_symbols = [symbol for symbol in symbols if symbol.endswith('USDT')]
    # print(usdt_symbols)
    
    
    # Use ThreadPoolExecutor to fetch data in parallel
    # with concurrent.futures.ThreadPoolExecutor() as executor:
    #   price_and_volume_data = list(executor.map(get_price_and_volume, usdt_symbols))
    
    # # Calculate market cap
    # market_caps = [calculate_market_cap(symbol, price, volume) for symbol, price, volume in price_and_volume_data]

    # # Filter out None values and create DataFrame
    # valid_market_caps = [(symbol, cap) for symbol, cap in market_caps if cap is not None]
    
    # # Create a DataFrame and save to CSV
    # df = pd.DataFrame(valid_market_caps, columns=['Symbol', 'Market Cap'])
    # df.to_csv('market_caps.csv', index=False)
    
    # print("Market caps calculated and saved to market_caps.csv")

if __name__ == "__main__":
    main()

NameError: name 'process_symbol' is not defined

## use pandas to filter  coins with marketcap =<  100 Million Dollars

In [23]:
df = pd.read_csv("market_caps.csv")

small_cap_coins_filter  = df[df['Market Cap'] <= 100_000_000]

small_cap_coins_filter.to_csv('small_cap_coins_filter.csv', index=False)
